## Train model

In [ ]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train a candidate XGBoost model
candidate_model = GradientBoostingClassifier().fit(X_train, y_train)

# train a baseline dummy model
baseline_model = DummyClassifier(strategy="uniform").fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["label"] = y_test

## Set Trubrics DataContext

In [ ]:
data_context = DataContext(
    target="label", testing_data=eval_data
)

## Build validations with Trubrics

In [ ]:
BUILD = False

In [ ]:
if BUILD:
    model_validator = ModelValidator(data=data_context, model=candidate_model)
    validations = [
        model_validator.validate_performance_against_threshold(metric="accuracy", threshold=0.8),
        model_validator.validate_performance_against_threshold(metric="recall", threshold=0.61, severity="warning"),
    ]

    rich.print(validations[0], validations[1])

    trubric = Trubric(
        name="mlflow-demo",
        data_context_name=data_context.name,
        data_context_version=data_context.version,
        validations=validations
    )

    trubric.save_local(path="./my_first_trubric.json")

## Execute the trubric from file

The `TrubricRun` object allows you to pull in a Trubric from file, and run that directly against any model and DataContext.

In [ ]:
from trubrics.validations.run import TrubricRun

In [ ]:
trubric_from_file = Trubric.parse_file("./my_first_trubric.json")

trubric_run_context = TrubricRun(
    data_context=data_context,
    model=candidate_model,
    trubric=trubric_from_file,
    tags=["nb-demo-new"],
    failing_severity="error",
)

## Save to mlflow

In [ ]:
with mlflow.start_run() as run:
    candidate_model_uri = mlflow.sklearn.log_model(candidate_model, "candidate_model").model_uri
    mlflow.evaluate(
        candidate_model_uri,
        eval_data,
        targets="label",
        model_type="classifier",
        evaluators="trubrics",
        evaluator_config={"trubric_run": trubric_run_context}
    )